In [1]:
# from google.colab import drive
# drive.flush_and_unmount()           # ignore errors if already unmounted

#If cannot remount, simply delete the mounted drive and then remount
# rm -rf /content/drive


In [2]:
# Colab cell
from google.colab import drive

drive.mount('/content/drive', force_remount=True)



Mounted at /content/drive


In [3]:
# Adjust these two for YOUR repo
REPO_OWNER = "kadkins3880"
REPO_NAME  = "STAT4160"   # e.g., unified-stocks-team1
BASE_DIR   = "/content/drive/MyDrive/dspt25"
CLONE_DIR  = f"{BASE_DIR}/{REPO_NAME}"
REPO_URL   = f"https://github.com/{REPO_OWNER}/{REPO_NAME}.git"

# if on my office computer

# REPO_NAME  = "lectureNotes"   # e.g., on my office computer
# BASE_DIR = r"E:\OneDrive - Auburn University Montgomery\teaching\AUM\STAT 4160 Productivity Tools" # on my office computer
# CLONE_DIR  = f"{BASE_DIR}\{REPO_NAME}"

import os, pathlib
pathlib.Path(BASE_DIR).mkdir(parents=True, exist_ok=True)


In [4]:
import os, subprocess, shutil, pathlib

if not pathlib.Path(CLONE_DIR).exists():
    !git clone {REPO_URL} {CLONE_DIR}
else:
    # If the folder exists, just ensure it's a git repo and pull latest
    os.chdir(CLONE_DIR)
    # !git status
    # !git pull --rebase # !git pull --ff-only
os.chdir(CLONE_DIR)
print("Working dir:", os.getcwd())

Working dir: /content/drive/MyDrive/dspt25/STAT4160


## Session 16 — Classical baselines

### Learning goals

By the end of class, students can:

1.  Fit a **per‑ticker** lags‑only linear regressor to predict **next‑day log return** $r_{t+1}$.
2.  Evaluate models with **MAE, sMAPE, MASE** using the **rolling‑origin splits** (with embargo) from Session 15.
3.  Log results in a **consistent table schema** for per‑ticker and split‑level summaries.
4.  Understand **ARIMA** at a glance and its common pitfalls (optional demo).

------------------------------------------------------------------------

## Agenda

-   where classical models fit; pitfalls with ARIMA; cross‑sectional regressors
-   results table schema & comparison to baselines
-   **In‑class lab**: train per‑ticker **Linear (lags‑only)** → evaluate across 2 splits → compare to naive/seasonal‑naive → log CSVs
-   Wrap‑up + homework brief

------------------------------------------------------------------------

### Why “classical” now?

-   Creates a **credible, strong baseline** against naive that’s still transparent.
-   Supports **fast iteration** and helps you debug feature definitions before deep models.

### Lags‑only linear regressor

-   **Features** at time $t$: `lag1`, `lag2`, `lag3` (i.e., past returns), optionally a few stable stats (`roll_std_20`, `zscore_20`).
-   **Target**: `r_1d` (next‑day log return).
-   Fit **per ticker** to avoid cross‑sectional leakage for now.

### ARIMA 60‑second pitfall tour

-   Stationarity: **fit on returns**, not prices (unless differencing).
-   Evaluation: **re‑fit only on train**; generate **one‑step‑ahead** forecasts on val, updating state **without peeking**.
-   Over‑differencing & mis‑specified seasonal terms → bad bias.
-   Computational cost grows with grid search; keep demo tiny.

### Results table schema (consistent across sessions)

-   **Per‑split summary**: `split, train_range, val_range, model, macro_mae, macro_smape, macro_mase, micro_mae, micro_smape, micro_mase`
-   **Per‑ticker metrics**: `split, ticker, n, model, mae, smape, mase`

------------------------------------------------------------------------

## In‑class lab (Colab‑friendly)


# 0) Load data (with fallbacks)

In [5]:
import os, pathlib, numpy as np, pandas as pd
from pathlib import Path

for p in ["data/raw","data/processed","reports","models","scripts","tests"]:
    Path(p).mkdir(parents=True, exist_ok=True)
print("Working dir:", os.getcwd())


# Load returns; if missing, synthesize
rpath = Path("data/processed/returns.parquet")
if rpath.exists():
    returns = pd.read_parquet(rpath)
else:
    rng = np.random.default_rng(0)
    dates = pd.bdate_range("2022-01-03", periods=360)
    rows=[]
    for t in ["AAPL","MSFT","GOOGL","AMZN","NVDA"]:
        eps = rng.normal(0,0.012, size=len(dates)).astype("float32")
        adj = 100*np.exp(np.cumsum(eps))
        df = pd.DataFrame({
            "date": dates, "ticker": t,
            "adj_close": adj.astype("float32"),
            "log_return": np.r_[np.nan, np.diff(np.log(adj))].astype("float32")
        })
        df["r_1d"] = df["log_return"].shift(-1) # target: next-day's return
        df["weekday"] = df["date"].dt.weekday.astype("int8")
        df["month"]   = df["date"].dt.month.astype("int8")
        rows.append(df)
    returns = pd.concat(rows, ignore_index=True).dropna().reset_index(drop=True)
    returns["ticker"] = returns["ticker"].astype("category")
    returns.to_parquet(rpath, index=False)

# Load features_v1 or derive minimal lags from returns if missing
fpath = Path("data/processed/features_v1.parquet")
if fpath.exists():
    feats = pd.read_parquet(fpath)
else:
    # Minimal lags derived just from returns
    feats = returns.sort_values(["ticker","date"]).copy()
    for k in [1,2,3]:
        feats[f"lag{k}"] = feats.groupby("ticker", observed= True)["log_return"].shift(k)
    feats = feats.dropna(subset=["lag1","lag2","lag3","r_1d"]).reset_index(drop=True)

# Harmonize
feats["date"] = pd.to_datetime(feats["date"])
feats["ticker"] = feats["ticker"].astype("category")
feats = feats.sort_values(["ticker","date"]).reset_index(drop=True)
feats.head()

Working dir: /content/drive/MyDrive/dspt25/STAT4160


,date,ticker,log_return,r_1d,weekday,month,lag1,lag2,lag3,roll_mean_20,roll_std_20,zscore_20,ewm_mean_20,ewm_std_20,exp_mean,exp_std,adj_close,volume
0,2020-01-29,AAPL,-0.018417,-0.002351,2,1,-0.012895,-0.019012,-0.004576,-0.004086,0.008476,-1.690830,-0.005252,0.009304,-0.004086,0.008476,92.154846,1598707
1,2020-01-30,AAPL,-0.002351,-0.012675,3,1,-0.018417,-0.012895,-0.019012,-0.004353,0.008324,0.240455,-0.004976,0.008875,-0.004003,0.008270,91.938454,2992900
2,2020-01-31,AAPL,-0.012675,0.002713,4,1,-0.002351,-0.018417,-0.012895,-0.004849,0.008517,-0.918756,-0.005709,0.008745,-0.004397,0.008280,90.780533,634335
3,2020-02-03,AAPL,0.002713,0.001568,0,2,-0.012675,-0.002351,-0.018417,-0.004268,0.008622,0.809695,-0.004907,0.008690,-0.004088,0.008224,91.027122,913454
4,2020-02-04,AAPL,0.001568,-0.001869,1,2,0.002713,-0.012675,-0.002351,-0.003963,0.008719,0.634254,-0.004291,0.008486,-0.003852,0.008126,91.169922,662663


# 1) Rolling‑origin date splits (reuse Session 15 logic)

In [6]:
def make_rolling_origin_splits(dates, train_min=252, val_size=63, step=63, embargo=5):
    u = np.array(sorted(pd.to_datetime(pd.Series(dates).unique())))
    splits=[]; i = train_min-1; n=len(u)
    while True:
        if i>=n: break
        a,b = u[0], u[i]; vs=i+embargo+1; ve=vs+val_size-1
        if ve>=n: break
        splits.append((a,b,u[vs],u[ve])); i+=step
    return splits

splits = make_rolling_origin_splits(feats["date"], 252, 63, 63, 5)
len(splits), splits[:2]

(0, [])

### [YW]: No splits generated from above. Check the data length and starting and ending date, adjust `train_min`, `val_size`, `step`, `embargo`, if necessary.

In [7]:
len(feats['date']), feats['date'].min(), feats['date'].max()

(3975, Timestamp('2020-01-29 00:00:00'), Timestamp('2020-09-07 00:00:00'))

In [8]:


# show per-ticker length
feats.groupby('ticker', observed=True)['date'].count().head(2)

,date
ticker,
AAPL,159
AMZN,159


### [YW] Rerun the splits with adjusted parameters.

In [9]:
splits = make_rolling_origin_splits(feats["date"], 80, 21, 21, 5)
len(splits), splits[:3]

(3,
 [(Timestamp('2020-01-29 00:00:00'),
   Timestamp('2020-05-19 00:00:00'),
   Timestamp('2020-05-27 00:00:00'),
   Timestamp('2020-06-24 00:00:00')),
  (Timestamp('2020-01-29 00:00:00'),
   Timestamp('2020-06-17 00:00:00'),
   Timestamp('2020-06-25 00:00:00'),
   Timestamp('2020-07-23 00:00:00')),
  (Timestamp('2020-01-29 00:00:00'),
   Timestamp('2020-07-16 00:00:00'),
   Timestamp('2020-07-24 00:00:00'),
   Timestamp('2020-08-21 00:00:00'))])

# 2) Metrics & baselines (from Session 15)

In [10]:
def mae(y, yhat):
    y = np.asarray(y); yhat = np.asarray(yhat);
    return float(np.mean(np.abs(y - yhat)))

def smape(y, yhat, eps=1e-8):
    y = np.asarray(y); yhat = np.asarray(yhat)
    return float(np.mean(2.0*np.abs(y - yhat)/(np.abs(y)+np.abs(yhat)+eps)))

def mase(y_true, y_pred, y_train_true, y_train_naive):
    scale = mae(y_train_true, y_train_naive) + 1e-12
    return float(mae(y_true, y_pred)/scale)

def add_baseline_preds(df: pd.DataFrame, seasonality:int=5) -> pd.DataFrame: # can handle sesonality
    out = df.copy()
    out["yhat_naive"] = out.groupby("ticker",observed = True)["log_return"].transform(lambda s: s)
    out["yhat_s"] = out.groupby("ticker",observed = True)["log_return"].transform(lambda s: s.shift(seasonality-1)) if seasonality>1 else out["yhat_naive"]
    return out

# **Per‑ticker lags‑only LinearRegression** (fit only on each split’s TRAIN)

Pay attention to the funciton `fit_predict_line_lags()`: standardize data + train + predict pipeline

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

# Choose features (lags only for in-class lab)
XCOLS = [c for c in ["lag1","lag2","lag3"] if c in feats.columns]
assert XCOLS, "No lag features found. Ensure features_v1 or fallback creation ran."

def fit_predict_lin_lags(train_df, val_df):
    """Fit per-ticker pipeline(StandardScaler, LinearRegression) on TRAIN; predict on VAL."""
    preds=[]
    for tkr, tr in train_df.groupby("ticker", observed = True):
        va = val_df[val_df["ticker"]==tkr]
        if len(tr)==0 or len(va)==0:
            continue
        pipe = Pipeline([("scaler", StandardScaler(with_mean=True, with_std=True)),
                         ("lr", LinearRegression())])
        pipe.fit(tr[XCOLS].values, tr["r_1d"].values)
        yhat = pipe.predict(va[XCOLS].values)
        out = va[["date","ticker","r_1d","log_return"]].copy()
        out["yhat_linlags"] = yhat.astype("float32")
        preds.append(out)
    return pd.concat(preds, ignore_index=True) if preds else pd.DataFrame(columns=["date","ticker","r_1d","log_return","yhat_linlags"])

# Evaluate **across the first 2 splits**; compare to naive/seasonal‑naive


In [12]:
seasonality = 5
feats_baseline = add_baseline_preds(feats, seasonality=seasonality)

def per_ticker_metrics(df_val_pred, df_train, method_col): # method_col: either yhat_naive or yat_s
    rows=[]
    for tkr, gv in df_val_pred.groupby("ticker", observed = True):
        if method_col not in gv:
            continue
        gv = gv.dropna(subset=["r_1d", method_col])
        if len(gv)==0:
            continue
        # TRAIN scale for MASE
        gt = df_train[df_train["ticker"]==tkr].dropna(subset=["r_1d"])
        gt_naive = gt["log_return"] if "yhat_s" not in method_col else gt["log_return"].shift(seasonality-1)
        gt_naive = gt_naive.loc[gt.index]
        rows.append({
            "ticker": tkr,
            "n": int(len(gv)),
            "mae": mae(gv["r_1d"], gv[method_col]),
            "smape": smape(gv["r_1d"], gv[method_col]),
            "mase": mase(gv["r_1d"], gv[method_col], gt["r_1d"], gt_naive),
        })
    return pd.DataFrame(rows)

def summarize_split(feats_frame, sid, split, save_prefix="linlags"):
    """
    sid: split id
    """
    a,b,c,d = split
    tr = feats_frame[(feats_frame["date"]>=a)&(feats_frame["date"]<=b)].copy()
    va = feats_frame[(feats_frame["date"]>=c)&(feats_frame["date"]<=d)].copy()
    # Predictions
    val_pred = fit_predict_lin_lags(tr, va)
    # Attach baseline preds on val slice
    va_base = add_baseline_preds(va, seasonality=seasonality)
    val_pred = val_pred.merge(va_base[["date","ticker","yhat_naive","yhat_s"]], on=["date","ticker"], how="left")

    # Per-ticker metrics
    pt_lin = per_ticker_metrics(val_pred, tr, "yhat_linlags"); pt_lin["model"] = "lin_lags"

    pt_nav = per_ticker_metrics(val_pred, tr, "yhat_naive");    pt_nav["model"] = "naive"
    pt_sea = per_ticker_metrics(val_pred, tr, "yhat_s");        pt_sea["model"] = f"s{seasonality}"

    # Save per-ticker
    out_pt = pd.concat([pt_lin.assign(split=sid), pt_nav.assign(split=sid), pt_sea.assign(split=sid)], ignore_index=True)
    out_pt.to_csv(f"reports/{save_prefix}_per_ticker_split{sid}.csv", index=False)

    # Aggregate
    def agg(df):
        if df.empty:
            return {"macro_mae":np.nan,"macro_smape":np.nan,"macro_mase":np.nan,"micro_mae":np.nan,"micro_smape":np.nan,"micro_mase":np.nan}
        macro = df[["mae","smape","mase"]].mean().to_dict()
        w = df["n"].to_numpy()
        micro = {"micro_mae": float(np.average(df["mae"], weights=w)),  # weighted average
                 "micro_smape": float(np.average(df["smape"], weights=w)),
                 "micro_mase": float(np.average(df["mase"], weights=w))}
        return {f"macro_{k}":float(v) for k,v in macro.items()} | micro

    rows=[]
    for name, pt in [("lin_lags", pt_lin), ("naive", pt_nav), (f"s{seasonality}", pt_sea)]:
        rows.append({"split":sid, "train_range": f"{a.date()}→{b.date()}",
                     "val_range": f"{c.date()}→{d.date()}",
                     "model":name, **agg(pt)})  # **agg(pt) unpacking the dict regurned by agg(pt)
    return pd.DataFrame(rows)

# Run on first 2 splits in class
summary_frames=[]
for sid, split in enumerate(splits[:2], start=1):
    sf = summarize_split(feats_baseline, sid, split, save_prefix="linlags")
    summary_frames.append(sf)

summary = pd.concat(summary_frames, ignore_index=True)
summary.to_csv("reports/linlags_summary_splits12.csv", index=False)
summary

,split,train_range,val_range,model,macro_mae,macro_smape,macro_mase,micro_mae,micro_smape,micro_mase
0,1,2020-01-29→2020-05-19,2020-05-27→2020-06-24,lin_lags,0.007942,1.628802,0.739563,0.007942,1.628802,0.739563
1,1,2020-01-29→2020-05-19,2020-05-27→2020-06-24,naive,0.011265,1.482772,1.052322,0.011265,1.482772,1.052322
2,1,2020-01-29→2020-05-19,2020-05-27→2020-06-24,s5,0.010848,1.481744,NaN,0.010848,1.481744,NaN
3,2,2020-01-29→2020-06-17,2020-06-25→2020-07-23,lin_lags,0.008499,1.667600,0.780760,0.008499,1.667600,0.780760
4,2,2020-01-29→2020-06-17,2020-06-25→2020-07-23,naive,0.011894,1.491683,1.091209,0.011894,1.491683,1.091209
5,2,2020-01-29→2020-06-17,2020-06-25→2020-07-23,s5,0.011245,1.432179,NaN,0.011245,1.432179,NaN


#  5) (Optional) Tiny ARIMA demo on **one ticker** for the first split

## ARIMA Model

**ARIMA** = **AutoRegressive Integrated Moving Average**

It’s one of the most widely used **time series forecasting models**, especially for univariate data (one variable evolving over time).

It combines three ideas:

| Component               | Meaning                                                      | Purpose                     |
| ----------------------- | ------------------------------------------------------------ | --------------------------- |
| **AR (AutoRegressive)** | Future value depends on **past values** of the series itself | captures persistence/trends |
| **I (Integrated)**      | The data are **differenced** to remove non-stationarity      | handles trends / drift      |
| **MA (Moving Average)** | Future value depends on **past forecast errors (residuals)** | smooths noise / shocks      |

ARIMA is denoted as
$$
\text{ARIMA}(p, d, q)
$$

where:

| Symbol | Interpretation                                                                            |
| ------ | ----------------------------------------------------------------------------------------- |
| (p)    | order of the **autoregressive** part (number of past observations used)                   |
| (d)    | degree of **differencing** (how many times you difference the data to make it stationary) |
| (q)    | order of the **moving average** part (number of past errors used)                         |

---

## The mathematical form

After differencing the series (d) times, define (y_t) as the stationary version.

Then ARIMA(p,d,q) satisfies:

$$
y_t = c + \sum_{i=1}^{p}\phi_i y_{t-i} + \sum_{j=1}^{q}\theta_j \varepsilon_{t-j} + \varepsilon_t
$$

where

* $ \phi_i $ = AR coefficients
* $ \theta_j $ = MA coefficients
* $ \varepsilon_t $ = white noise (random error)

If $d > 0$, then$(y_t = (1 - B)^d X_t$, where $B$ is the backshift operator ($B X_t = X_{t-1})$.

---

## Step-by-step workflow

1. **Check stationarity**
   Plot or use tests (ADF, KPSS).
   If non-stationary → difference once or more.

2. **Identify p and q**

   * Look at the **ACF** (autocorrelation) and **PACF** (partial autocorrelation) plots.
   * Sharp cutoff in ACF → MA part; sharp cutoff in PACF → AR part.

3. **Fit the model**
   Estimate parameters $c, \phi_i, \theta_j$.

4. **Diagnose residuals**
   Check if residuals look like white noise.

5. **Forecast**
   Use the fitted model to project future values.

---

##  Example in Python

```python
import pandas as pd
import statsmodels.api as sm

# Example: monthly airline passengers dataset
data = sm.datasets.airpassengers.load_pandas().data
y = data['value']

# Fit ARIMA(1,1,1)
model = sm.tsa.ARIMA(y, order=(1,1,1))
result = model.fit()

print(result.summary())

# Forecast next 12 periods
forecast = result.forecast(steps=12)
forecast.plot(title="ARIMA(1,1,1) Forecast")
```

---

## Extensions of ARIMA

| Model                                   | Meaning                         | Handles                                    |
| --------------------------------------- | ------------------------------- | ------------------------------------------ |
| **SARIMA(p,d,q)(P,D,Q,s)**              | Seasonal ARIMA                  | seasonal cycles                            |
| **ARIMAX / SARIMAX**                    | ARIMA with exogenous regressors | external variables (e.g. temperature, GDP) |
| **VAR / VARIMA**                        | Vector (multivariate) version   | multiple related time series               |
| **ARIMA with Box–Cox / log transforms** | Nonlinear variance stabilizing  | heteroskedasticity                         |

---

## TL;DR

ARIMA tries to predict today’s value as:

> “A weighted average of **past values** and **past mistakes**, after removing long-term trend.”

So it blends **trend correction (I)**, **momentum (AR)**, and **shock absorption (MA)** in one unified model.




In [13]:
# Optional: quick ARIMA(1,0,0) demo predicting r_{t+1} on val for a single ticker
try:
    from statsmodels.tsa.arima.model import ARIMA
    import warnings; warnings.filterwarnings("ignore") # ignore (suppress) all warning messages
    a,b,c,d = splits[0]
    tkr = feats["ticker"].cat.categories[0] # pick the first ticker
    tr = feats[(feats["ticker"]==tkr) & (feats["date"]>=a) & (feats["date"]<=b)]
    va = feats[(feats["ticker"]==tkr) & (feats["date"]>=c) & (feats["date"]<=d)]
    # Fit on TRAIN returns only (endog = log_return). Predict one-step ahead for VAL dates.  endog: endogenous variable (observed, dependent var). cf exog: exogenous variable (external, independent var)
    model = ARIMA(tr["log_return"].to_numpy(), order=(1,0,0))
    res = model.fit()
    # Forecast length = len(va), one-step-ahead with dynamic=False updates internally
    # (For strict no-peek rolling one-step, loop and append val true values; here we keep demo simple.)
    fc = res.forecast(steps=len(va)) #note the model is recursive, using the previously predicted values to make predicitons for the next predict
    arima_mae = mae(va["r_1d"], fc)  # compare against next-day return
    float(arima_mae)
except Exception as e:
    print("ARIMA demo skipped:", e)

## Wrap‑up

-   You trained a **per‑ticker lags‑only linear** model and compared it fairly to **naive** and **seasonal‑naive** using the **same splits** and **MASE scale** (from the train window).
-   You logged results in a **stable schema** that you’ll reuse for future models (LSTM / Transformer).
-   ARIMA can be illustrative but is often **fragile + slower**; treat it as optional for your project scale.

------------------------------------------------------------------------

## Homework (due before next session)

**Goal:** 1) Run the linear lags baseline across **all splits**; 2) Write your **first model card** (Quarto) for the classical baseline.

### Part A — CLI script to evaluate Linear‑Lags across *all* splits


In [14]:
# save to scripts/eval_linlags.py
#!/usr/bin/env python
from __future__ import annotations
import argparse, numpy as np, pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from pathlib import Path

def mae(y,yhat): return float(np.mean(np.abs(np.asarray(y)-np.asarray(yhat))))
def smape(y,yhat,eps=1e-8):
    y = np.asarray(y); yhat = np.asarray(yhat)
    return float(np.mean(2*np.abs(y-yhat)/(np.abs(y)+np.abs(yhat)+eps)))
def mase(y_true, y_pred, y_train_true, y_train_naive):
    return float(mae(y_true, y_pred) / (mae(y_train_true, y_train_naive)+1e-12))

def make_splits(dates, train_min, val_size, step, embargo):
    u = np.array(sorted(pd.to_datetime(pd.Series(dates).unique())))
    splits=[]; i=train_min-1; n=len(u)
    while True:
        if i>=n: break
        a,b = u[0], u[i]; vs=i+embargo+1; ve=vs+val_size-1
        if ve>=n: break
        splits.append((a,b,u[vs],u[ve])); i+=step
    return splits

def add_baselines(df, seasonality):
    out = df.copy()
    out["yhat_naive"] = out.groupby("ticker", observed = True)["log_return"].transform(lambda s: s)
    out["yhat_s"] = out.groupby("ticker", observed = True)["log_return"].transform(lambda s: s.shift(seasonality-1)) if seasonality>1 else out["yhat_naive"]
    return out

def fit_predict_lin(train_df, val_df, xcols):
    from sklearn.linear_model import LinearRegression
    from sklearn.preprocessing import StandardScaler
    from sklearn.pipeline import Pipeline
    preds=[]
    for tkr, tr in train_df.groupby("ticker"):
        va = val_df[val_df["ticker"]==tkr]
        if len(tr)==0 or len(va)==0: continue
        pipe = Pipeline([("scaler", StandardScaler()), ("lr", LinearRegression())])
        pipe.fit(tr[xcols].values, tr["r_1d"].values)
        yhat = pipe.predict(va[xcols].values)
        out = va[["date","ticker","r_1d","log_return"]].copy()
        out["yhat_linlags"] = yhat
        preds.append(out)
    return pd.concat(preds, ignore_index=True) if preds else pd.DataFrame()

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--features", default="data/processed/features_v1.parquet")
    ap.add_argument("--seasonality", type=int, default=5)
    ap.add_argument("--train-min", type=int, default=252)
    ap.add_argument("--val-size", type=int, default=63)
    ap.add_argument("--step", type=int, default=63)
    ap.add_argument("--embargo", type=int, default=5)
    ap.add_argument("--xcols", nargs="+", default=["lag1","lag2","lag3"])
    ap.add_argument("--out-summary", default="reports/linlags_summary.csv")
    ap.add_argument("--out-per-ticker", default="reports/linlags_per_ticker_split{sid}.csv")
    # args = ap.parse_args() # notworking in Colab
    args, unknown = ap.parse_known_args() # fix
    print("Parsed args:", args)


    df = pd.read_parquet(args.features).sort_values(["ticker","date"]).reset_index(drop=True)
    df["ticker"] = df["ticker"].astype("category")
    splits = make_splits(df["date"], args.train_min, args.val_size, args.step, args.embargo)
    df = add_baselines(df, args.seasonality)

    rows=[]
    for sid, (a,b,c,d) in enumerate(splits, start=1):
        tr = df[(df["date"]>=a)&(df["date"]<=b)]
        va = df[(df["date"]>=c)&(df["date"]<=d)]
        val_pred = fit_predict_lin(tr, va, args.xcols)
        va = va.merge(val_pred[["date","ticker","yhat_linlags"]], on=["date","ticker"], how="left")
        # per-ticker
        pts=[]
        for tkr, gv in va.groupby("ticker"):
            gv = gv.dropna(subset=["r_1d","yhat_linlags"])
            if len(gv)==0: continue
            gt = tr[tr["ticker"]==tkr].dropna(subset=["r_1d"])
            gt_naive = gt["log_return"]  # scale comparator for MASE
            pts.append({"ticker":tkr,"n":int(len(gv)),
                        "mae": mae(gv["r_1d"], gv["yhat_linlags"]),
                        "smape": smape(gv["r_1d"], gv["yhat_linlags"]),
                        "mase": mase(gv["r_1d"], gv["yhat_linlags"], gt["r_1d"], gt_naive)})
        pt = pd.DataFrame(pts)
        Path("reports").mkdir(exist_ok=True)
        pt.assign(split=sid, model="lin_lags").to_csv(args.out_per_ticker.format(sid=sid), index=False)

        # aggregate
        if not pt.empty:
            macro = pt[["mae","smape","mase"]].mean().to_dict()
            w = pt["n"].to_numpy()
            micro = {"micro_mae": float(np.average(pt["mae"], weights=w)),
                     "micro_smape": float(np.average(pt["smape"], weights=w)),
                     "micro_mase": float(np.average(pt["mase"], weights=w))}
        else:
            macro = {"mae":np.nan,"smape":np.nan,"mase":np.nan}
            micro = {"micro_mae":np.nan,"micro_smape":np.nan,"micro_mase":np.nan}
        rows.append({"split":sid,"train_range":f"{a.date()}→{b.date()}","val_range":f"{c.date()}→{d.date()}",
                     "model":"lin_lags", "macro_mae":float(macro["mae"]), "macro_smape":float(macro["smape"]), "macro_mase":float(macro["mase"]),
                     **micro})

    pd.DataFrame(rows).to_csv(args.out_summary, index=False)
    print("Wrote", args.out_summary)

if __name__ == "__main__":
    main()

Parsed args: Namespace(features='data/processed/features_v1.parquet', seasonality=5, train_min=252, val_size=63, step=63, embargo=5, xcols=['lag1', 'lag2', 'lag3'], out_summary='reports/linlags_summary.csv', out_per_ticker='reports/linlags_per_ticker_split{sid}.csv')
Wrote reports/linlags_summary.csv


# Run the script form Shell

In [15]:
%%bash
chmod +x scripts/eval_linlags.py
python scripts/eval_linlags.py --xcols lag1 lag2 lag3

Parsed args: Namespace(features='data/processed/features_v1.parquet', seasonality=5, train_min=252, val_size=63, step=63, embargo=5, xcols=['lag1', 'lag2', 'lag3'], out_summary='reports/linlags_summary.csv', out_per_ticker='reports/linlags_per_ticker_split{sid}.csv')
Wrote reports/linlags_summary.csv


### Part B — Quarto **Model Card** for the Linear‑Lags baseline

Create `reports/model_card_linear.qmd`:

````markdown
---
title: "Model Card — Linear Lags (Per‑Ticker)"
format:
  html:
    theme: cosmo
    toc: true
params:
  model_name: "Linear Lags (per‑ticker)"
  data: "features_v1.parquet"
---
> **Educational use only — not trading advice.** Predicts next‑day log return $r_{t+1}$ using past lags.

## Overview

- **Model:** Per‑ticker linear regression with features: `lag1`, `lag2`, `lag3`.
- **Data:** `features_v1.parquet` (Session 10).  
- **Splits:** Expanding, quarterly val, 5‑day embargo (Session 15).  
- **Baselines:** Naive and seasonal‑naive \(s=5\).

## Metrics (across splits)

```{python}
import pandas as pd
df = pd.read_csv("reports/linlags_summary.csv")
df
```

## Discussion

* **Assumptions:** Linear relation to recent returns; stationarity at return level.
* **Strengths:** Fast, interpretable, leakage‑resistant with proper splits.
* **Failure modes:** Regime shifts; volatility spikes; nonlinearity.
* **Ethics:** Educational; not suitable for trading.

````

 We first install Quarto in Colab (see lec3-inclass.ipynb)

In [16]:
# Install Quarto CLI (one-time per Colab runtime)
# !wget -q https://quarto.org/download/latest/quarto-linux-amd64.deb -O /tmp/quarto.deb
# !dpkg -i /tmp/quarto.deb || apt-get -y -f install >/dev/null && dpkg -i /tmp/quarto.deb
# !quarto --version

#Alternatively, save it to G-drive, and only need to download the first time. The size of  quarto-linux-amd64.deb is ~125Mb.
# Path to store the deb package
deb_path = "/content/drive/MyDrive/quarto-linux-amd64.deb"

# Download only if not already saved
!test -f $deb_path || wget -q https://quarto.org/download/latest/quarto-linux-amd64.deb -O $deb_path

# Install from Drive (fast, no re-download)
!dpkg -i $deb_path || apt-get -y -f install >/dev/null && dpkg -i $deb_path #-f: fix package dependency issues
!quarto --version

Selecting previously unselected package quarto.
(Reading database ... 121703 files and directories currently installed.)
Preparing to unpack .../MyDrive/quarto-linux-amd64.deb ...
Unpacking quarto (1.7.34) ...
Setting up quarto (1.7.34) ...
(Reading database ... 125814 files and directories currently installed.)
Preparing to unpack .../MyDrive/quarto-linux-amd64.deb ...
Unpacking quarto (1.7.34) over (1.7.34) ...
Setting up quarto (1.7.34) ...
1.7.34


Render (if Quarto is available).

In [17]:
!find . -type f -name "linlags_summary*.csv"


./reports/linlags_summary.csv
./reports/linlags_summary_splits12.csv
./STAT4160/reports/linlags_summary_splits12.csv
./STAT4160/reports/linlags_summary.csv


In [18]:
!quarto render reports/model_card_linear.qmd --output-dir reports/

ERROR: Book chapter 'index.qmd' not found

Stack trace:
    at throwInputNotFound (file:///opt/quarto/bin/quarto.js:100735:19)
    at findInputs (file:///opt/quarto/bin/quarto.js:100766:17)
    at findChapters (file:///opt/quarto/bin/quarto.js:100778:19)
    at bookRenderItems (file:///opt/quarto/bin/quarto.js:100781:11)
    at Object.bookProjectConfig [as config] (file:///opt/quarto/bin/quarto.js:100686:31)
    at eventLoopTick (ext:core/01_core.js:175:7)
    at async projectContext (file:///opt/quarto/bin/quarto.js:82364:38)
    at async render (file:///opt/quarto/bin/quarto.js:91794:19)
    at async Command.actionHandler (file:///opt/quarto/bin/quarto.js:91975:32)
    at async Command.execute (file:///opt/quarto/bin/quarto.js:8253:13)


#Part C — Quick test to safeguard results shape

In [19]:
# save tests/test_linlags_results.py
import pandas as pd, os

def test_linlags_summary_exists_and_columns():
    assert os.path.exists("reports/linlags_summary.csv")
    df = pd.read_csv("reports/linlags_summary.csv")
    need = {"split","model","macro_mae","micro_mae"}
    assert need.issubset(df.columns)

In [20]:
%%bash
pytest -q -k linlags_results  # the test doesnot pass

F                                                                        [100%]
=================================== FAILURES ===================================
___________________ test_linlags_summary_exists_and_columns ____________________

    def test_linlags_summary_exists_and_columns():
        assert os.path.exists("reports/linlags_summary.csv")
>       df = pd.read_csv("reports/linlags_summary.csv")
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

tests/test_linlags_results.py:6: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py:1026: in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py:620: in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
/usr/local/lib/python3.12/dist-packages/pandas/io

CalledProcessError: Command 'b'pytest -q -k linlags_results  # the test doesnot pass\n'' returned non-zero exit status 1.

# Part D — (Optional) Extend features or add Ridge

* Try `--xcols lag1 lag2 lag3 roll_std_20 zscore_20` (if present in `features_v1`).
* Swap `LinearRegression` for `Ridge(alpha=1.0)`; log and compare.

In [21]:
%%bash
chmod +x scripts/eval_linlags.py
python scripts/eval_linlags.py --xcols lag1 lag2 lag3 roll_std_20 zscore_20

Parsed args: Namespace(features='data/processed/features_v1.parquet', seasonality=5, train_min=252, val_size=63, step=63, embargo=5, xcols=['lag1', 'lag2', 'lag3', 'roll_std_20', 'zscore_20'], out_summary='reports/linlags_summary.csv', out_per_ticker='reports/linlags_per_ticker_split{sid}.csv')
Wrote reports/linlags_summary.csv


### Add Redge (L^2 regression) to the function `fit_predict_lin()`

In [22]:
def fit_predict_lin(train_df, val_df, xcols):
    from sklearn.linear_model import LinearRegression, Ridge
    from sklearn.preprocessing import StandardScaler
    from sklearn.pipeline import Pipeline
    import pandas as pd

    preds = []
    for tkr, tr in train_df.groupby("ticker"):
        va = val_df[val_df["ticker"] == tkr]
        if len(tr) == 0 or len(va) == 0:
            continue

        # --- Linear Regression ---
        pipe_lin = Pipeline([
            ("scaler", StandardScaler()),
            ("lr", LinearRegression())
        ])
        pipe_lin.fit(tr[xcols].values, tr["r_1d"].values)
        yhat_lin = pipe_lin.predict(va[xcols].values)

        # --- Ridge Regression ---
        pipe_ridge = Pipeline([
            ("scaler", StandardScaler()),
            ("ridge", Ridge(alpha=1.0))
        ])
        pipe_ridge.fit(tr[xcols].values, tr["r_1d"].values)
        yhat_ridge = pipe_ridge.predict(va[xcols].values)

        # --- Combine outputs ---
        out = va[["date", "ticker", "r_1d", "log_return"]].copy()
        out["yhat_linlags"] = yhat_lin        # ordinary least squares
        out["yhat_ridge"] = yhat_ridge        # L2-regularized version
        preds.append(out)

    return pd.concat(preds, ignore_index=True) if preds else pd.DataFrame()
